In [1]:
import os
import sys
import time
from datetime import datetime
from IPython.display import display, HTML

# Asegurarnos que la ruta del código está en el path
project_root = "C:\\Users\\emers\\Documents\\SUNASS_CAMI_CENTRO\\DEV_SUNASS\\NewsAgent"
code_dir = os.path.join(project_root, "codigo")
if code_dir not in sys.path:
    sys.path.append(code_dir)

#codigo de testeo
ruta = os.listdir(code_dir)
for archivo in ruta:
    print(f"ruta encontrada: {archivo}")


#Importar utilidades específicas para notebooks
from notebook_utils import setup_environment

# Mostrar un mensaje de inicio
print("Iniciando ejecución del orquestador...")

# Función para ejecutar el orquestador
def ejecutar_orquestador(fecha, prompt="detallado", batch_size=3, pause_seconds=60):
    """
    Ejecuta el orquestador para una fecha específica.
    
    Args:
        fecha (str): Fecha en formato DDMMYYYY
        prompt (str): Tipo de prompt (simple, detallado, estructurado, anti-ruido)
        batch_size (int): Tamaño del lote de imágenes
        pause_seconds (int): Segundos de pausa entre lotes
    """
    # Configurar el entorno y logging antes de importar main
    logger = setup_environment(project_root)
    
    # Ahora importamos run_pipeline (después de configurar el logging)
    from main import run_pipeline
    
    # Configurar el archivo config.yaml
    config_file = os.path.join(project_root, "config.yaml")
    try:
        # Guardar configuración existente si existe
        config_backup = None
        if os.path.exists(config_file):
            with open(config_file, 'r', encoding='utf-8') as f:
                config_backup = f.read()
        
        # Escribir nueva configuración
        with open(config_file, 'w', encoding='utf-8') as f:
            f.write(f"""api:
  prompt_key: "{prompt}"
  batch_size: {batch_size}
  pause_seconds: {pause_seconds}
""")
        
        print(f"Configuración aplicada: prompt={prompt}, batch_size={batch_size}, pause={pause_seconds}s")
        
        # Ejecutar el orquestador
        print(f"Ejecutando orquestador para fecha: {fecha}")
        print("Esto puede tomar varios minutos. Por favor, espera...")
        
        start_time = datetime.now()
        run_pipeline(custom_date_str=fecha)
        end_time = datetime.now()
        
        duration = (end_time - start_time).total_seconds()
        print(f"✅ Procesamiento completado en {duration:.2f} segundos")
        
        # Restaurar configuración original si existía
        if config_backup:
            with open(config_file, 'w', encoding='utf-8') as f:
                f.write(config_backup)
    
    except Exception as e:
        print(f"Error ejecutando orquestador: {str(e)}")
        import traceback
        traceback.print_exc()

# Ejemplo de uso - descomenta la línea que necesites:
# ejecutar_orquestador("01042025")                      # Fecha básica con valores predeterminados
# ejecutar_orquestador("01042025", prompt="simple")     # Cambiando el prompt
# ejecutar_orquestador("01042025", batch_size=2)        # Procesando sólo 2 imágenes antes de pausar

print("Script cargado. Descomenta y ejecuta una de las líneas de ejemplo o usa la función 'ejecutar_orquestador'.")

ruta encontrada: lib
ruta encontrada: main.py
ruta encontrada: NewsAg.ipynb
ruta encontrada: notebook_utils.py
Iniciando ejecución del orquestador...
Script cargado. Descomenta y ejecuta una de las líneas de ejemplo o usa la función 'ejecutar_orquestador'.


In [3]:
# Ejemplo de ejecución:
ejecutar_orquestador("01042025")

ModuleNotFoundError: No module named 'frontend'

## Panel de control para ejecución fácil

Para facilitar aún más el uso de esta herramienta a personas no familiarizadas con Python, puedes utilizar el siguiente panel interactivo:

In [14]:
#Descomenta y ejecuta esto para usar el panel interactivo
import ipywidgets as widgets
from IPython.display import display, clear_output

# Crear widgets para la entrada de datos
fecha_input = widgets.Text(value='19032025', description='Fecha (DDMMYYYY):')
prompt_dropdown = widgets.Dropdown(
    options=['simple', 'detallado', 'estructurado', 'anti-ruido'],
    value='estructurado',
    description='Tipo de prompt:'
)
batch_slider = widgets.IntSlider(value=3, min=1, max=10, description='Batch size:')
pause_slider = widgets.IntSlider(value=60, min=10, max=300, description='Pausa (seg):')

# Función para manejar el botón de ejecución
def on_ejecutar_button_clicked(b):
    with output:
        clear_output()
        ejecutar_orquestador(
            fecha_input.value,
            prompt=prompt_dropdown.value,
            batch_size=batch_slider.value,
            pause_seconds=pause_slider.value
        )

# Crear botón de ejecución
ejecutar_button = widgets.Button(description='Ejecutar Orquestador')
ejecutar_button.on_click(on_ejecutar_button_clicked)

# Crear área de salida
output = widgets.Output()

# Mostrar interfaz
display(widgets.VBox([
    widgets.HBox([fecha_input, prompt_dropdown]),
    widgets.HBox([batch_slider, pause_slider]),
    ejecutar_button,
    output
]))